In [1]:
import tensorflow as tf
import numpy as np
import random as rd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
image = np.loadtxt('bbs-train.txt')
label = np.loadtxt('label-train.txt')
# initialize the dataset
image_train = image[:4500]
image_val = image[4500:]
labels = np.zeros((len(label),2))
for i in range(len(label)):
    if label[i,1] == 0:
        labels[i] = [0.0, 1.0]
    if label[i,1] == 1:
        labels[i] = [1.0, 0.0]

label_train = labels[:4500]
label_val = labels[4500:]
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [3]:
x = tf.placeholder(tf.float32, shape = [None,800])
y = tf.placeholder(tf.float32, shape= [None,2])

W = tf.Variable(tf.zeros([800,2]))
b = tf.Variable(tf.zeros([2]))

y_ = tf.matmul(x, W) + b

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
## [1, 2, 2, 1]  1: input 像素 1 represents channel  1: represent output channel
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# first layer
## The convolution will compute 32 features for each 5x5 patch
## The first two dimensions are the patch size, the next is the number of input channels, 
## and the last is the number of output channels
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# reshape
## the second and third dimensions corresponding to image width and height, 
## and the final dimension corresponding to the number of color channels.
x_image = tf.reshape(x, [-1, 40, 20, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## now the dimension will reduce to 10 * 5  and 64 features, and we give 1024 nueron for 
## fully connected neuron network
W_fc1 = weight_variable([10 * 5 * 64, 1024])
b_fc1 = bias_variable([1024])

## finally flatten the input into fully connected neuron network
h_pool2_flat = tf.reshape(h_pool2, [-1, 10 * 5 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## add our softmax layer for regression above
W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))

optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# *******************************************************************
# DONT Run it here or it will crash the computer. Run it on unix env.
# *******************************************************************
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch_x, batch_y = next_batch(50, image_train, label_train)
        if i%100 == 0:
            # for every step except for the output in training process, we wond't use dropout 
            train_accuracy = accuracy.eval(feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
            print("step %d, training accuracy %g"%(i, train_accuracy))

        # without output, we just use dropout to train the data
        optimizer.run(feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})

    print("training complete!")
    print('test accuracy %g' % accuracy.eval(feed_dict={x: image_val, y: label_val, keep_prob: 1.0}))
    y_pred = sess.run(y_conv, feed_dict={x: image_val, keep_prob: 1.0})
    print(y_pred)

step 0, training accuracy 0.58
step 100, training accuracy 0.38
step 200, training accuracy 0.74
step 300, training accuracy 0.78
step 400, training accuracy 0.68
step 500, training accuracy 0.9
step 600, training accuracy 0.86
step 700, training accuracy 0.68
step 800, training accuracy 0.84
step 900, training accuracy 0.62
step 1000, training accuracy 0.7
step 1100, training accuracy 0.9
step 1200, training accuracy 0.72
step 1300, training accuracy 0.9
step 1400, training accuracy 0.84
step 1500, training accuracy 0.96
step 1600, training accuracy 0.92
step 1700, training accuracy 0.8
step 1800, training accuracy 0.8
step 1900, training accuracy 0.86
training complete!
test accuracy 0.978261
[[ -5.19537170e+02   1.33170410e+02]
 [ -3.43377856e+03   2.99168213e+03]
 [ -1.98657312e+03   2.08800195e+03]
 [ -1.66722791e+03   1.33688843e+03]
 [ -7.03347107e+02   5.94187195e+02]
 [ -1.92773743e+03   2.36374756e+03]
 [ -4.50813171e+02   4.92204285e+01]
 [ -6.76065979e+02   4.14269196e+02]


In [31]:
for i in range(46):
    if data[i,0] > data[i,1]:
        y_pred[i,0] = 1.0
        y_pred[i,1] = 0.0
    else:
        y_pred[i,0] = 0.0
        y_pred[i,1] = 1.0


In [33]:
np.equal(y_pred, label_val)

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False, False],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ Tr

array([[  1.00000000e+00,   1.33170410e+02],
       [ -3.43377856e+03,   2.99168213e+03],
       [ -1.98657312e+03,   2.08800195e+03],
       [ -1.66722791e+03,   1.33688843e+03],
       [ -7.03347107e+02,   5.94187195e+02],
       [ -1.92773743e+03,   2.36374756e+03],
       [ -4.50813171e+02,   4.92204285e+01],
       [ -6.76065979e+02,   4.14269196e+02],
       [ -4.18180566e+03,   5.05023779e+03],
       [ -5.90607227e+03,   6.26868604e+03],
       [ -1.04770374e+03,   1.19799988e+03],
       [ -2.12502954e+03,   1.96726404e+03],
       [ -3.71199768e+02,   2.08359894e+02],
       [  1.00000000e+00,   0.00000000e+00],
       [ -4.03263770e+03,   3.06119995e+03],
       [ -1.41826746e+03,   1.20061694e+03],
       [ -2.13480054e+03,   2.11607275e+03],
       [ -4.43193756e+02,   1.46650610e+01],
       [ -3.81311987e+03,   3.17364355e+03],
       [ -8.60236511e+02,   9.44475403e+02],
       [ -1.37349475e+03,   1.32875256e+03],
       [ -2.95196313e+03,   1.66381360e+03],
       [ -